In [48]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import sys
import os
import pickle
import torch
from IPython.display import HTML


# Allow imports from the parent directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from main import tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
fit_file_path = 'datas/datas_fit.pkl'

with open(fit_file_path, 'rb') as f:

    datas_fit = pickle.load(f)
    
data = datas_fit[0]

In [51]:
#indices = torch.randperm(data.m)[:400]
#f = data.plot_vel_data(indices = indices, scale = 3e-4, frame = 1, gif = True)
#HTML(f.to_html5_video())

In [53]:
indices = torch.randperm(data.m)[:data.m]
f = data.plot_vel(indices = indices, scale = 8, frame=0, color="blue", gif = True)
HTML(f.to_html5_video())
